In [1]:
!pip -q install pandas numpy requests tqdm chembl_webresource_client
!mkdir -p data results


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 5.9 MB/s eta 0:00:00


In [2]:
import pandas as pd

# Change path if yours is in results/
biomarkers_path = "/content/integrated_biomarker_panel.csv"
df = pd.read_csv(biomarkers_path)

# Prefer biomarkers supported by BOTH omics, then fall back if needed
if "source" in df.columns:
    targets = df[df["source"] == "RNA&PROT"]["feature"].dropna().astype(str).unique().tolist()
else:
    targets = df["feature"].dropna().astype(str).unique().tolist()

# Keep a manageable list for APIs (you can increase later)
targets = targets[:100]
print("Targets loaded:", len(targets))
print("Example targets:", targets[:10])


Targets loaded: 8
Example targets: ['STARD13', 'HSPA12B', 'KANK3', 'CDH5', 'RTKN2', 'JAM2', 'MMRN2', 'WWC2']


In [3]:
import requests
from tqdm import tqdm

OT_GQL = "https://api.platform.opentargets.org/api/v4/graphql"

# A broad lung cancer disease term in Open Targets often maps to EFO IDs.
# We'll query by disease name first, then use the returned disease ID.
def find_disease_id(disease_name="lung carcinoma"):
    query = """
    query($q: String!) {
      search(queryString: $q, entityNames: ["disease"], page: {index: 0, size: 5}) {
        hits { id name entity }
      }
    }
    """
    r = requests.post(OT_GQL, json={"query": query, "variables": {"q": disease_name}}, timeout=60)
    r.raise_for_status()
    hits = r.json()["data"]["search"]["hits"]
    return hits

hits = find_disease_id("lung carcinoma")
hits


[{'id': 'EFO_0001071', 'name': 'lung carcinoma', 'entity': 'disease'},
 {'id': 'EFO_0003060',
  'name': 'non-small cell lung carcinoma',
  'entity': 'disease'},
 {'id': 'EFO_0000708',
  'name': 'squamous cell lung carcinoma',
  'entity': 'disease'},
 {'id': 'EFO_0003050',
  'name': 'large cell lung carcinoma',
  'entity': 'disease'},
 {'id': 'EFO_0000702',
  'name': 'small cell lung carcinoma',
  'entity': 'disease'}]

In [4]:
import requests
import pandas as pd
from tqdm import tqdm

OT_GQL = "https://api.platform.opentargets.org/api/v4/graphql"

def gql(query, variables):
    r = requests.post(OT_GQL, json={"query": query, "variables": variables}, timeout=60)
    # If it errors, print server message for debugging
    if r.status_code != 200:
        try:
            print("Status:", r.status_code)
            print("Response:", r.text[:500])
        except Exception:
            pass
        r.raise_for_status()
    return r.json()

# 1) Find a lung cancer disease ID (EFO)
def find_disease_candidates(q="lung cancer"):
    query = """
    query($q: String!) {
      search(queryString: $q, entityNames: ["disease"], page: {index: 0, size: 10}) {
        hits { id name entity }
      }
    }"""
    return gql(query, {"q": q})["data"]["search"]["hits"]

disease_hits = find_disease_candidates("lung carcinoma")
print("Top disease hits:")
for h in disease_hits[:5]:
    print(h)

# Pick the first hit unless you want to choose another
LUNG_DISEASE_ID = disease_hits[0]["id"]
print("\nUsing disease id:", LUNG_DISEASE_ID, "-", disease_hits[0]["name"])

# 2) Convert gene symbol -> Ensembl target id (Open Targets target id is Ensembl gene id)
def symbol_to_ensembl(symbol: str):
    query = """
    query($q: String!) {
      search(queryString: $q, entityNames: ["target"], page: {index: 0, size: 5}) {
        hits { id name entity }
      }
    }"""
    hits = gql(query, {"q": symbol})["data"]["search"]["hits"]
    for h in hits:
        # target IDs typically look like ENSG000001...
        if h["entity"] == "target" and str(h["id"]).startswith("ENSG"):
            return h["id"], h.get("name", "")
    return None, None

# 3) Get association score via target->associatedDiseases without filters (then search within)
#    This avoids the common "filters" argument mismatch causing 400s.
def get_target_disease_score_unfiltered(ensembl_id: str, disease_id: str):
    query = """
    query($ensemblId: String!) {
      target(ensemblId: $ensemblId) {
        id
        approvedSymbol
        associatedDiseases(page: {index: 0, size: 500}) {
          rows {
            disease { id name }
            score
          }
        }
      }
    }"""
    data = gql(query, {"ensemblId": ensembl_id})
    t = data.get("data", {}).get("target", None)
    if not t:
        return None
    rows = t["associatedDiseases"]["rows"] if t.get("associatedDiseases") else []
    # Find the specific disease in returned rows
    for r in rows:
        if r["disease"]["id"] == disease_id:
            return {
                "ensembl_id": ensembl_id,
                "approvedSymbol": t.get("approvedSymbol", ""),
                "disease_id": r["disease"]["id"],
                "disease_name": r["disease"]["name"],
                "ot_score": r["score"],
            }
    # If not found, score = 0
    return {
        "ensembl_id": ensembl_id,
        "approvedSymbol": t.get("approvedSymbol", ""),
        "disease_id": disease_id,
        "disease_name": "",
        "ot_score": 0.0,
    }

# ---- RUN on your targets list ----
ot_rows = []
bad = []

for sym in tqdm(targets):  # assumes you already created `targets` from your biomarker file
    try:
        ens, _ = symbol_to_ensembl(sym)
        if not ens:
            bad.append((sym, "no_ensembl_match"))
            continue

        rec = get_target_disease_score_unfiltered(ens, LUNG_DISEASE_ID)
        if rec is None:
            bad.append((sym, "no_target_record"))
            continue

        ot_rows.append({
            "gene_symbol": sym,
            "ensembl_id": rec["ensembl_id"],
            "ot_approved_symbol": rec["approvedSymbol"],
            "lung_disease_id": rec["disease_id"],
            "lung_disease_name": rec["disease_name"],
            "ot_score": float(rec["ot_score"]),
        })
    except Exception as e:
        bad.append((sym, f"error: {e}"))
        continue

ot_df = pd.DataFrame(ot_rows).sort_values("ot_score", ascending=False)
ot_df.to_csv("results/opentargets_lung_scores.csv", index=False)

print("\n✅ Saved results/opentargets_lung_scores.csv")
print("Top rows:")
display(ot_df.head(10))

print("\nSkipped/failed symbols:", len(bad))
print("Examples:", bad[:10])


Top disease hits:
{'id': 'EFO_0001071', 'name': 'lung carcinoma', 'entity': 'disease'}
{'id': 'EFO_0003060', 'name': 'non-small cell lung carcinoma', 'entity': 'disease'}
{'id': 'EFO_0000708', 'name': 'squamous cell lung carcinoma', 'entity': 'disease'}
{'id': 'EFO_0003050', 'name': 'large cell lung carcinoma', 'entity': 'disease'}
{'id': 'EFO_0000702', 'name': 'small cell lung carcinoma', 'entity': 'disease'}

Using disease id: EFO_0001071 - lung carcinoma


100%|██████████| 8/8 [00:02<00:00,  3.10it/s]


✅ Saved results/opentargets_lung_scores.csv
Top rows:


,gene_symbol,ensembl_id,ot_approved_symbol,lung_disease_id,lung_disease_name,ot_score
4,RTKN2,ENSG00000182010,RTKN2,EFO_0001071,lung carcinoma,0.039515
5,JAM2,ENSG00000154721,JAM2,EFO_0001071,lung carcinoma,0.025209
3,CDH5,ENSG00000179776,CDH5,EFO_0001071,lung carcinoma,0.017568
7,WWC2,ENSG00000151718,WWC2,EFO_0001071,lung carcinoma,0.015198
0,STARD13,ENSG00000133121,STARD13,EFO_0001071,lung carcinoma,0.009613
1,HSPA12B,ENSG00000132622,HSPA12B,EFO_0001071,lung carcinoma,0.007392
2,KANK3,ENSG00000186994,KANK3,EFO_0001071,,0.000000
6,MMRN2,ENSG00000173269,MMRN2,EFO_0001071,,0.000000



Skipped/failed symbols: 0
Examples: []


In [5]:
from chembl_webresource_client.new_client import new_client

chembl_target = new_client.target
chembl_drug = new_client.drug
chembl_mech = new_client.mechanism

# Use top N biologically relevant targets by Open Targets score
top_targets = ot_df.sort_values("ot_score", ascending=False).head(30)["gene_symbol"].tolist()
print("Top targets:", top_targets[:10])

def get_chembl_targets_for_symbol(symbol: str):
    # Search ChEMBL targets by preferred name containing symbol (best-effort)
    res = chembl_target.filter(pref_name__icontains=symbol).only(["target_chembl_id","pref_name","target_type"])
    return list(res)

def get_drugs_for_chembl_target(tchembl: str):
    # Mechanisms often link drugs to targets
    res = chembl_mech.filter(target_chembl_id=tchembl).only(["molecule_chembl_id","mechanism_of_action","action_type","direct_interaction"])
    return list(res)

def get_drug_info(mol_chembl_id: str):
    # Try to see if it's an approved drug record
    res = chembl_drug.filter(molecule_chembl_id=mol_chembl_id).only(["molecule_chembl_id","pref_name","max_phase","first_approval"])
    return list(res)

rows = []
for sym in tqdm(top_targets):
    tlist = get_chembl_targets_for_symbol(sym)
    for t in tlist[:5]:
        tchembl = t["target_chembl_id"]
        mechs = get_drugs_for_chembl_target(tchembl)
        for m in mechs:
            mol = m["molecule_chembl_id"]
            dinfo = get_drug_info(mol)
            if dinfo:
                d = dinfo[0]
                rows.append({
                    "gene_symbol": sym,
                    "target_chembl_id": tchembl,
                    "chembl_target_name": t.get("pref_name",""),
                    "molecule_chembl_id": mol,
                    "drug_name": d.get("pref_name",""),
                    "max_phase": d.get("max_phase", None),
                    "first_approval": d.get("first_approval", None),
                    "action_type": m.get("action_type",""),
                    "mechanism_of_action": m.get("mechanism_of_action",""),
                    "direct_interaction": m.get("direct_interaction", None),
                })

chembl_map = pd.DataFrame(rows).drop_duplicates()
chembl_map.head(10)


Top targets: ['RTKN2', 'JAM2', 'CDH5', 'WWC2', 'STARD13', 'HSPA12B', 'KANK3', 'MMRN2']


100%|██████████| 8/8 [00:06<00:00,  1.21it/s]


""


In [6]:
chembl_map.to_csv("results/chembl_target_drug_map.csv", index=False)
print("Saved results/chembl_target_drug_map.csv")


Saved results/chembl_target_drug_map.csv


In [8]:
print("chembl_map shape:", chembl_map.shape)
print("chembl_map columns:", list(chembl_map.columns))

print("\not_df shape:", ot_df.shape)
print("ot_df columns:", list(ot_df.columns))

chembl_map.head()


chembl_map shape: (0, 0)
chembl_map columns: []

ot_df shape: (8, 6)
ot_df columns: ['gene_symbol', 'ensembl_id', 'ot_approved_symbol', 'lung_disease_id', 'lung_disease_name', 'ot_score']


""


In [14]:
!pip -q install chembl_webresource_client pandas tqdm


In [15]:
import pandas as pd
from tqdm import tqdm
from chembl_webresource_client.new_client import new_client

chembl_target = new_client.target
chembl_mech   = new_client.mechanism
chembl_drug   = new_client.drug


In [16]:
def chembl_targets_from_gene_symbol(symbol, max_targets=10):
    """
    Version-safe mapping:
    - fetch targets
    - inspect target_components
    - match gene_symbol manually
    """
    out = []
    res = chembl_target.filter(target_type__in=["SINGLE PROTEIN", "PROTEIN FAMILY"]).only([
        "target_chembl_id", "pref_name", "target_type", "target_components"
    ])

    for t in res:
        comps = t.get("target_components", [])
        for c in comps:
            if c.get("gene_symbol") == symbol:
                out.append({
                    "target_chembl_id": t["target_chembl_id"],
                    "pref_name": t.get("pref_name", ""),
                    "target_type": t.get("target_type", "")
                })
        if len(out) >= max_targets:
            break

    return out


In [17]:
def drugs_for_target(target_chembl_id, max_mechs=200):
    mechs = chembl_mech.filter(target_chembl_id=target_chembl_id).only([
        "molecule_chembl_id", "mechanism_of_action", "action_type", "direct_interaction"
    ])[:max_mechs]

    rows = []
    for m in mechs:
        mol = m.get("molecule_chembl_id")
        if not mol:
            continue

        dinfo = chembl_drug.filter(molecule_chembl_id=mol).only([
            "molecule_chembl_id", "pref_name", "max_phase", "first_approval"
        ])

        if not dinfo:
            continue

        d = dinfo[0]
        rows.append({
            "molecule_chembl_id": mol,
            "drug_name": d.get("pref_name", ""),
            "max_phase": d.get("max_phase", 0),
            "first_approval": d.get("first_approval", None),
            "action_type": m.get("action_type", ""),
            "mechanism_of_action": m.get("mechanism_of_action", ""),
            "direct_interaction": m.get("direct_interaction", False),
        })
    return rows


In [19]:
!pip -q install pandas tqdm requests

import time
import requests
import pandas as pd
from tqdm import tqdm
from urllib.parse import quote

BASE = "https://www.ebi.ac.uk/chembl/api/data"
session = requests.Session()
session.headers.update({"User-Agent": "colab-omics-drug-repurposing/1.0"})


In [20]:
def get_json_with_retry(url, params=None, max_retries=6, base_sleep=1.5):
    for attempt in range(max_retries):
        try:
            r = session.get(url, params=params, timeout=60)
            # retry on rate limit or server error
            if r.status_code in (429, 500, 502, 503, 504):
                sleep = base_sleep * (2 ** attempt)
                print(f"Retry {attempt+1}/{max_retries} for {r.status_code} -> sleeping {sleep:.1f}s")
                time.sleep(sleep)
                continue
            r.raise_for_status()
            return r.json()
        except Exception as e:
            sleep = base_sleep * (2 ** attempt)
            print(f"Exception: {e} -> sleeping {sleep:.1f}s")
            time.sleep(sleep)
    return None


In [21]:
def chembl_target_search(symbol, limit=5):
    # /target/search?q=EGFR
    url = f"{BASE}/target/search.json"
    data = get_json_with_retry(url, params={"q": symbol, "limit": limit})
    if not data:
        return []
    return data.get("targets", [])

def chembl_mechanisms_for_target(target_chembl_id, limit=200):
    url = f"{BASE}/mechanism.json"
    data = get_json_with_retry(url, params={"target_chembl_id": target_chembl_id, "limit": limit})
    if not data:
        return []
    return data.get("mechanisms", [])

def chembl_drug_info(molecule_chembl_id):
    url = f"{BASE}/drug.json"
    data = get_json_with_retry(url, params={"molecule_chembl_id": molecule_chembl_id, "limit": 1})
    if not data:
        return None
    drugs = data.get("drugs", [])
    return drugs[0] if drugs else None


In [24]:
import requests, pandas as pd
from tqdm import tqdm

OT_GQL = "https://api.platform.opentargets.org/api/v4/graphql"

def gql(query, variables):
    r = requests.post(OT_GQL, json={"query": query, "variables": variables}, timeout=60)
    r.raise_for_status()
    return r.json()

def find_disease_id(query_str="lung carcinoma"):
    q = """
    query($q: String!) {
      search(queryString: $q, entityNames: ["disease"], page: {index: 0, size: 5}) {
        hits { id name entity }
      }
    }"""
    hits = gql(q, {"q": query_str})["data"]["search"]["hits"]
    return hits[0]["id"], hits[0]["name"], hits

def symbol_to_ensembl(symbol: str):
    q = """
    query($q: String!) {
      search(queryString: $q, entityNames: ["target"], page: {index: 0, size: 5}) {
        hits { id name entity }
      }
    }"""
    hits = gql(q, {"q": symbol})["data"]["search"]["hits"]
    for h in hits:
        if h["entity"] == "target" and str(h["id"]).startswith("ENSG"):
            return h["id"], h.get("name","")
    return None, None

def target_disease_score_unfiltered(ensembl_id: str, disease_id: str):
    q = """
    query($ensemblId: String!) {
      target(ensemblId: $ensemblId) {
        id
        approvedSymbol
        associatedDiseases(page: {index: 0, size: 500}) {
          rows { disease { id name } score }
        }
      }
    }"""
    data = gql(q, {"ensemblId": ensembl_id})
    t = data.get("data", {}).get("target")
    if not t:
        return 0.0, ""
    rows = t["associatedDiseases"]["rows"] if t.get("associatedDiseases") else []
    for r in rows:
        if r["disease"]["id"] == disease_id:
            return float(r["score"]), r["disease"]["name"]
    return 0.0, ""

disease_id, disease_name, hits = find_disease_id("lung carcinoma")
print("Using disease:", disease_name, disease_id)

ot_rows = []
for sym in tqdm(targets_200):  # your 92 targets list
    ens, _ = symbol_to_ensembl(sym)
    if not ens:
        continue
    score, dname = target_disease_score_unfiltered(ens, disease_id)
    ot_rows.append({"gene_symbol": sym, "ensembl_id": ens, "lung_disease_id": disease_id,
                    "lung_disease_name": dname, "ot_score": score})

ot_df_full = pd.DataFrame(ot_rows).sort_values("ot_score", ascending=False)
ot_df_full.to_csv("results/opentargets_lung_scores_full.csv", index=False)
print("✅ Saved results/opentargets_lung_scores_full.csv")
display(ot_df_full.head(10))


Using disease: lung carcinoma EFO_0001071


100%|██████████| 92/92 [00:31<00:00,  2.89it/s]

✅ Saved results/opentargets_lung_scores_full.csv


,gene_symbol,ensembl_id,lung_disease_id,lung_disease_name,ot_score
32,PTPRB,ENSG00000127329,EFO_0001071,lung carcinoma,0.375146
27,EPAS1,ENSG00000116016,EFO_0001071,lung carcinoma,0.284540
45,QKI,ENSG00000112531,EFO_0001071,lung carcinoma,0.281676
69,ERG,ENSG00000157554,EFO_0001071,lung carcinoma,0.281454
17,TAL1,ENSG00000162367,EFO_0001071,lung carcinoma,0.278109
47,LMO2,ENSG00000135363,EFO_0001071,lung carcinoma,0.277185
71,CAV1,ENSG00000105974,EFO_0001071,lung carcinoma,0.045948
4,RTKN2,ENSG00000182010,EFO_0001071,lung carcinoma,0.039515
77,LIMCH1,ENSG00000064042,EFO_0001071,lung carcinoma,0.039174
65,AGER,ENSG00000204305,EFO_0001071,lung carcinoma,0.038698


In [25]:
import pandas as pd
import numpy as np

# Load files
chembl_map = pd.read_csv("results/chembl_target_drug_map.csv")
ot_df_full = pd.read_csv("results/opentargets_lung_scores_full.csv")

# Merge Open Targets scores
merged = chembl_map.merge(
    ot_df_full[["gene_symbol", "ot_score"]].drop_duplicates(),
    on="gene_symbol",
    how="left"
)

# -----------------------------
# 🚫 REMOVE KNOWN BAD MAPPINGS
# -----------------------------
BAD_GENES = {"ERG"}  # add more if you spot obvious mismatches
merged = merged[~merged["gene_symbol"].isin(BAD_GENES)]

# -----------------------------
# Clean and score
# -----------------------------
merged["ot_score"] = pd.to_numeric(merged["ot_score"], errors="coerce").fillna(0.0)
merged["max_phase"] = pd.to_numeric(merged["max_phase"], errors="coerce").fillna(0.0)
merged["direct_interaction"] = merged["direct_interaction"].fillna(False).astype(int)

merged["repurpose_score"] = (
    0.5 * merged["ot_score"] +
    0.4 * (merged["max_phase"] / 4.0) +
    0.1 * merged["direct_interaction"]
)

# Rank final candidates
top = (
    merged.sort_values("repurpose_score", ascending=False)
    .dropna(subset=["drug_name"])
    .drop_duplicates(subset=["drug_name", "gene_symbol"])
    .head(50)
)

# Save final results
top.to_csv("results/top_repurposing_candidates_scored.csv", index=False)

print("✅ Saved results/top_repurposing_candidates_scored.csv")
display(
    top[
        ["drug_name", "gene_symbol", "ot_score", "max_phase",
         "repurpose_score", "mechanism_of_action"]
    ].head(15)
)


✅ Saved results/top_repurposing_candidates_scored.csv


,drug_name,gene_symbol,ot_score,max_phase,repurpose_score,mechanism_of_action
